In [1]:
import pandas as pd

train=pd.read_csv('train.csv',index_col=0)

train.head()

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707 entries, 0 to 706
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sudden_fever           707 non-null    float64
 1   headache               707 non-null    float64
 2   mouth_bleed            707 non-null    float64
 3   nose_bleed             707 non-null    float64
 4   muscle_pain            707 non-null    float64
 5   joint_pain             707 non-null    float64
 6   vomiting               707 non-null    float64
 7   rash                   707 non-null    float64
 8   diarrhea               707 non-null    float64
 9   hypotension            707 non-null    float64
 10  pleural_effusion       707 non-null    float64
 11  ascites                707 non-null    float64
 12  gastro_bleeding        707 non-null    float64
 13  swelling               707 non-null    float64
 14  nausea                 707 non-null    float64
 15  chills

In [3]:
from sklearn.preprocessing import LabelEncoder

train['prognosis']=LabelEncoder().fit_transform(train.prognosis)

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers


In [5]:
data = train.values


In [6]:
max_vals = np.max(data[:, :-1], axis=0)
min_vals = np.min(data[:, :-1], axis=0)
data[:, :-1] = (data[:, :-1] - min_vals) / (max_vals - min_vals)


In [7]:
# Define the generator network:

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(data.shape[1], activation='tanh'))
    return model



In [8]:
# Define the discriminator network:

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(1024, input_shape=(data.shape[1],)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1))
    return model

In [9]:
# Define the loss functions:

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [10]:
# Define the optimizer:

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [11]:
# Define the training loop:
def train_step(generator, discriminator, real_data, noise_dim):
    noise = tf.random.normal([len(real_data), noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_data = generator(noise, training=True)

        real_output = discriminator(real_data, training=True)
        fake_output = discriminator(fake_data, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(generator, discriminator, data, epochs, batch_size, noise_dim):
    for epoch in range(epochs):
        for i in range(0, len(data), batch_size):
            real_data = data[i:i+batch_size]
            train_step(generator, discriminator, real_data, noise_dim)
        print(f'Epoch {epoch+1} completed.')


In [12]:
# Train the GAN: give the code after this
# Define the noise dimension
noise_dim = 100

# Build the generator and discriminator
generator = make_generator_model()
discriminator = make_discriminator_model()

# Train the GAN
train(generator, discriminator, data, epochs=100, batch_size=32, noise_dim=noise_dim)


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.
Epoch 6 completed.
Epoch 7 completed.
Epoch 8 completed.
Epoch 9 completed.
Epoch 10 completed.
Epoch 11 completed.
Epoch 12 completed.
Epoch 13 completed.
Epoch 14 completed.
Epoch 15 completed.
Epoch 16 completed.
Epoch 17 completed.
Epoch 18 completed.
Epoch 19 completed.
Epoch 20 completed.
Epoch 21 completed.
Epoch 22 completed.
Epoch 23 completed.
Epoch 24 completed.
Epoch 25 completed.
Epoch 26 completed.
Epoch 27 completed.
Epoch 28 completed.
Epoch 29 completed.
Epoch 30 completed.
Epoch 31 completed.
Epoch 32 completed.
Epoch 33 completed.
Epoch 34 completed.
Epoch 35 completed.
Epoch 36 completed.
Epoch 37 completed.
Epoch 38 completed.
Epoch 39 completed.
Epoch 40 completed.
Epoch 41 completed.
Epoch 42 completed.
Epoch 43 completed.
Epoch 44 completed.
Epoch 45 completed.
Epoch 46 completed.
Epoch 47 completed.
Epoch 48 completed.
Epoch 49 completed.
Epoch 50 completed.
Epoch 51 

In [13]:
# Save the generator model
generator.save('generator_model.h5')


In [14]:
gen=pd.DataFrame(generated_data,columns=pd.read_csv('train.csv',index_col=0).columns)
gen.describe()

NameError: name 'generated_data' is not defined

In [16]:
# Define the list of labels
labels = pd.read_csv('train.csv',index_col=0).prognosis.value_counts().keys()

# Generate new data from the trained generator
num_samples = 1000
noise = tf.random.normal([num_samples, noise_dim])
generated_data = generator(noise, training=False)

# # Convert the generated data to discrete labels
# def map_to_labels(x):
#     index = int(x * len(labels))
#     return labels[index]

# generated_labels = [map_to_labels(x) for x in generated_data]

# Save the generated labels to a file
np.save('generated_labels.npy', generated_data)


In [18]:
data=np.load('generated_data.npy')
fdf=pd.DataFrame(data,columns=pd.read_csv('train.csv',index_col=0).columns)

In [20]:
fdf.prognosis.describe()

count    10000.000000
mean        -0.826050
std          0.365149
min         -1.000000
25%         -0.999725
50%         -0.991999
75%         -0.873819
max          0.962503
Name: prognosis, dtype: float64